This is me trying to integrate all of the things I've learned: 1. SVM, SVC, torch, torch, pandas, numpy

The objective is to build a SVM algorithm that takes the data set un compute using SVM kernel trick to find the best hyperplain.

# Importing and Reading the data

## Importing

In [1]:
import torch
import pandas as pd
import numpy as np
import file_operations
import os

## Reading

In [ ]:
current_dir = os.getcwd()
print(current_dir)
paths = file_operations.create_project_path("Project 6 SVM")
file_name = 'breast-cancer-wisconsin.data'
file_path = os.path.join(paths["data_dir"], file_name)
df = pd.read_csv(file_path)
display(df)
display(df.info())

d:\ImportanFiles\Coding Related\Repositories\Machine Learning project related\Project 6 SVM


TypeError: 'NoneType' object is not subscriptable

# Dataframe preprocessing

In [ ]:
df.replace('?', np.nan, inplace = True)
df.dropna(inplace = True)
df.drop(['id'], axis = 1, inplace = True)

In [ ]:
from sklearn import model_selection, svm
X = np.array(df.drop(['class'], axis = 1)).astype('float64')
y = np.array(df['class']).astype('float64')
# Change label from 2, 4 to -1, 1, with 1 meaning that the label is malignant -1 meaning that the label is benign.
y = np.where(y == 4, 1, np.where(y == 2, -1, y))

# Getting the scikit learn version of accuracy.

In [ ]:
%%time
sk_accuracy_sum = 0
count = 0
for count in range(0, 10000):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    sk_accuracy_sum += accuracy
    count += 1

print(count)
print(sk_accuracy_sum / count)

# Creating My Own GPU accelerated version(Binary Classification):

## Thoughts:
1. Checking the dimensions

In [ ]:
print(type(X))
print(X.shape)

2. The feature is 9 dimensional, so the hyper plain best seperating hyperplain should be 8 dimensional.
3. transfer numpy array into tensor stored on gpu.
4. Create a randomly shuffle function to shuffle the rows but not columns of X.
5. Create a SVC weights tensor for later train to train on. 
    The objective is to find the best weights, because SVC in this case is a normal equation of 8 dimensions, we need to find the coefficients for this equation. when found, the equation is the best seperating huperplain.
6. Randomize weights for the SVC
7. A function to find out a single n dimentional point to a (n-1) dimentional span's shortest vector, then multiply that vector with a classifier vector to determine wheather or not that they are reversed vectors with each other.
    This will help determine wheather a dot is on the best seperating heperplain's "positive side" or "negative side".

    Note that the first vector will be considered the "class 1" vector, meaning that it's relation to the best seperating hyperplain will be considered "Class 1"

    For example:
        if a dot is [1, 2], the line best seperating hyperplain is y = x, then the shortest vector from the dot to the hyper plain is [1.5, 1.5] - [1, 2] = [0.5, -0.5]
        the the direction of this vector [0.5, -0.5] will be considered "class 1". The vectors opposed to this vector will be considered "class 2"

## 3. Tranfer numpy into tensor stored on gpu.

### Checking availability

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Current device: {device.capitalize()}.")

### Transformation.

In [ ]:
X_gpu = torch.tensor(X, device = device)
y_gpu = torch.tensor(y, device = device)
print(X_gpu.shape)
print(y_gpu.shape)

## 4. Shuffle the tensors' row vectors.

In [ ]:
indices = torch.randperm(X_gpu.shape[0])
X_gpu = X_gpu[indices]

## 5. Find soft margin and SVC between each features.(Handling outliers and misclassification)

Equations for the SVC:

`Ax_1 + Bx_2 + Cx_3 + ... + Hx_8 + b = 0`

`W = [A, B, C, D, E, F, G, H]`

`X = [x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8]`

`b` is a constant

### Create a random 

In [ ]:
# print(X.shape)
# (683, 9)
# Randomizing a weight matrix to adjust.
SVC_weights = torch.rand(1, X.shape[1] - 1, dtype = torch.float64, device = device)
SVC_bias = torch.tensor(0.0, dtype=torch.float64, device=device)
print(SVC_weights.shape)
print(SVC_weights)
print(SVC_weights.dtype)
print(SVC_bias)

### Function to culculate the n dimensional direction vector.

Let's go through each point carefully with intuitive, geometric explanations.

---

### 1. What Does \( \omega^T x_0 + b \) Mean Geometrically?

Even though \( x_0 \) is generally not on the hyperplane, plugging it into the expression  
\[
\omega^T x_0 + b
\]
tells us something very important:

- **Projection on the Normal:**  
  Think of \( \omega \) as a vector perpendicular (normal) to the hyperplane defined by  
  \[
  \omega^T x + b = 0.
  \]
  When you compute \( \omega^T x_0 \), you are essentially projecting the point \( x_0 \) onto the direction of \( \omega \).

- **Offset by \( b \):**  
  The term \( b \) shifts the hyperplane away from the origin. So \( \omega^T x_0 + b \) measures how far along the direction of \( \omega \) the point \( x_0 \) lies relative to where the hyperplane is located.

- **Signed Distance (up to normalization):**  
  Although \( x_0 \) isn’t on the hyperplane, the sign of \( \omega^T x_0 + b \) tells you on which side of the hyperplane \( x_0 \) lies:
  - A positive value means it lies on the side of the hyperplane where the normal vector \( \omega \) points.
  - A negative value means it lies on the opposite side.
  
  The actual perpendicular (or shortest) distance from \( x_0 \) to the hyperplane is given by:
  \[
  d = \frac{\omega^T x_0 + b}{\|\omega\|}.
  \]
  Here, \( d \) is a scalar that indicates both the distance and the side of the hyperplane.

**Visualizing in 2D:**  
Imagine a line in the plane representing your hyperplane. The vector \( \omega \) is an arrow perpendicular to that line. Now, take any point \( x_0 \) in the plane. If you drop a perpendicular from \( x_0 \) to the line, the length of that perpendicular (and its sign) is determined by \( \omega^T x_0 + b \) once you normalize it by \( \|\omega\| \).

---

### 2. Your Proposed Algorithm for Classification

Your understanding is on the right track. Here’s the step-by-step geometric view:

1. **Random Initialization:**  
   - **Randomize a Normal Vector \( \omega \) and Bias \( b \).**  
     These define your initial hyperplane. \( \omega \) points perpendicular to your hyperplane, and \( b \) shifts the hyperplane relative to the origin.

2. **Calculate the Signed Distance \( d \):**  
   - For each feature vector (each training example) \( x_0 \), compute:
     \[
     d = \frac{\omega^T x_0 + b}{\|\omega\|}.
     \]
   - **Interpretation:**
     - If \( d > 0 \), the point is on one side of the hyperplane (say, Class 1).
     - If \( d < 0 \), it’s on the opposite side (say, Class 2).

3. **Classification Decision:**  
   - Simply check the sign of \( d \). This is equivalent to checking the sign of \( \omega^T x_0 + b \) (if you ignore the normalization for classification purposes).

This procedure captures the essence of how SVMs use geometry to separate classes.

---

### 3. Defining a Loss Function and Optimizing \( \omega \) and \( b \)

Once you have the function to compute \( d \) for each point, you need a way to adjust \( \omega \) and \( b \) so that the hyperplane best separates your data. This is where a loss function comes in. A popular choice is the **hinge loss**, which for a single sample is given by:
\[
L_i = \max\{0, 1 - y_i (\omega^T x_i + b)\},
\]
where \( y_i \) is the label, typically \( +1 \) or \( -1 \).

#### **Intuitive Explanation in 2D**

Imagine you have a set of points in the plane with labels:
- **Correct Classification with a Margin:**  
  For a correctly classified point (say, with \( y_i = +1 \)), you want:
  \[
  \omega^T x_i + b \ge 1.
  \]
  Similarly, for \( y_i = -1 \), you want:
  \[
  \omega^T x_i + b \le -1.
  \]
  The value “1” here defines a margin around the hyperplane.

- **Misclassified Points or Points within the Margin:**  
  If a point falls on the wrong side or within this margin, the hinge loss becomes positive, meaning you incur a penalty. This penalty encourages the model to adjust \( \omega \) and \( b \) to push these points further into the correct region.

#### **Using Gradient Descent**

- **Aggregate the Loss:**  
  Sum the hinge loss over all training examples (often also adding a regularization term like \( \frac{1}{2} \|\omega\|^2 \) to encourage a large margin):
  \[
  L = \frac{1}{2}\|\omega\|^2 + C \sum_{i=1}^{N} \max\{0, 1 - y_i (\omega^T x_i + b)\},
  \]
  where \( C \) is a hyperparameter controlling the trade-off between maximizing the margin and minimizing the classification error.

- **Gradient Descent Steps:**  
  1. **Compute the Gradient:**  
     For each misclassified or margin-violating point (i.e., where \( 1 - y_i (\omega^T x_i + b) > 0 \)), calculate the gradient of the loss with respect to \( \omega \) and \( b \).
     
     - The gradient with respect to \( \omega \) for a single point (ignoring regularization) is:
       \[
       -y_i x_i \quad \text{(if the point contributes to the loss)}.
       \]
     - Similarly, the gradient with respect to \( b \) is:
       \[
       -y_i.
       \]
     - Add the derivative of the regularization term \( \omega \) (from \( \frac{1}{2}\|\omega\|^2 \)) when updating \( \omega \).

  2. **Update Parameters:**  
     Use gradient descent updates:
     \[
     \omega \leftarrow \omega - \eta \cdot \text{(gradient w.r.t. } \omega\text{)},
     \]
     \[
     b \leftarrow b - \eta \cdot \text{(gradient w.r.t. } b\text{)},
     \]
     where \( \eta \) is the learning rate.

- **Geometric Interpretation in 2D:**  
  Picture your hyperplane (a line in 2D). For every point that is either misclassified or too close to the line (inside the margin), the loss function "pushes" the hyperplane away from that point:
  - If a positive point is too close or misclassified, the gradient will adjust the line so that the distance (as measured by \( \omega^T x_i + b \)) increases.
  - If a negative point is in the wrong region, the update will adjust the line in the opposite direction.
  
  As you iterate, the hyperplane shifts and rotates until the margin is maximized (points are well-separated) and misclassifications are minimized.

---

### Summary

1. **\( \omega^T x_0 + b \)** is the projection of the point \( x_0 \) onto the normal direction \( \omega \), shifted by \( b \). It tells you the signed distance (up to normalization) from \( x_0 \) to the hyperplane.
2. **Your Algorithm:**  
   - Randomize \( \omega \) and \( b \).
   - For each point, compute the signed distance \( d = \frac{\omega^T x_0 + b}{\|\omega\|} \).
   - Classify based on the sign of \( d \) (or equivalently, \( \omega^T x_0 + b \)).
3. **Loss Function and Optimization:**  
   - Use a loss function (like hinge loss) that penalizes misclassified points and those within the margin.
   - Sum the loss over your training set, add a regularization term, and use gradient descent to update \( \omega \) and \( b \).
   - In 2D, imagine adjusting a line so that all points are on the correct side and as far from the line as possible, which is exactly what the optimization process achieves.

This geometric perspective should help you visualize and implement the SVM training process using gradient descent in your PyTorch model.

In [ ]:
def cal_signed_distance(n_point, hyperplain_weights, hyperplain_bias):    
    # Check input valitility function HERE:
    """
    Compute the signed distance from each data point in X to the hyperplane defined by weights and bias.
    
    Parameters:
      X(n_point) (Tensor): A tensor of shape (n_samples, n_features) containing the feature vectors.
      hyperplain_weights (Tensor): A tensor of shape (1, n_features) representing the hyperplane's normal vector.
      hyperplain_bias (Tensor): A scalar tensor representing the hyperplane's bias.
    
    Returns:
      distances (Tensor): A tensor of shape (n_samples, 1) with the signed distances.
                          Positive means the point is on the side of the hyperplane pointed to by weights; negative means the opposite.
    """
    # Calculate the raw score for each sample:
    #   For each sample x: raw_score = weights * x^T + bias
    # This yields a tensor of shape (n_samples, 1).
    raw_scores = torch.matmul(n_point, hyperplain_weights.T) + hyperplain_bias
    
    # Compute the norm (magnitude) of the weights vector.
    weight_norm = torch.norm(hyperplain_weights)
    
    # Calculate the signed distance: divide the raw score by the norm.
    # This gives the perpendicular distance from the point to the hyperplane.
    distances = raw_scores / weight_norm
    return distances

In [ ]:
def hinge_loss(distances, labels, margin=1.0):
    """
    Compute the mean hinge loss for a set of predictions.
    
    Parameters:
      distances (Tensor): A tensor of shape (n_samples, 1) with signed distances from the hyperplane.
      labels (Tensor): A tensor of shape (n_samples,) with the true labels (+1 or -1).
      margin (float): The margin value used in the hinge loss (default is 1.0).
    
    Returns:
      loss (Tensor): A scalar tensor representing the mean hinge loss over the batch.
    """
    # Remove the extra dimension for convenience.
    distances = distances.squeeze()
    
    # Compute the hinge loss for each sample:
    #   loss_i = max(0, margin - label_i * distance_i)
    losses = torch.clamp(margin - labels * distances, min=0)
    
    # Return the average loss.
    return losses.mean()

def update_model(X, y, weights, bias, learning_rate = 0.01):
    """
    Perform one gradient descent update step for the SVM parameters.
    
    Parameters:
      X (Tensor): A tensor of shape (n_samples, n_features) with the input data.
      y (Tensor): A tensor of shape (n_samples,) with the true labels.
      weights (Tensor): A tensor of shape (1, n_features) representing the current weights.
      bias (Tensor): A scalar tensor representing the current bias.
      learning_rate (float): The learning rate for gradient descent.
    
    Returns:
      loss_value (float): The computed loss value for this batch.
      weights (Tensor): The updated weights.
      bias (Tensor): The updated bias.
    """
    # Enable gradient tracking on weights and bias.
    weights.requires_grad_(True)
    bias.requires_grad_(True)``
    
    # Compute the signed distances for all training samples.
    distances = cal_signed_distance(X, weights, bias)
    
    # Compute the hinge loss given the distances and true labels.
    loss = hinge_loss(distances, y)
    
    # Perform backpropagation to compute the gradients.
    loss.backward()
    
    # Update weights and bias using gradient descent.
    with torch.no_grad():
        weights -= learning_rate * weights.grad
        bias -= learning_rate * bias.grad
    
    # Zero out the gradients after the update.
    weights.grad.zero_()
    bias.grad.zero_()
    
    return loss.item(), weights, bias

# ----- Example Training Loop -----
num_epochs = 100
learning_rate = 0.01

for epoch in range(num_epochs):
    loss_value, SVC_weights, SVC_bias = update_model(X_gpu, y_gpu, SVC_weights, SVC_bias, learning_rate)
    print(f"Epoch {epoch + 1:03d} | Loss: {loss_value:.4f}")